In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import matplotlib.pyplot as plt
import seaborn

import os
import glob

import numpy as np
import math

from urllib import request, error
import json

import datetime, time
from datetime import datetime



import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import os
import plotly
import plotly.express as ex
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [2]:
from functions.Plots import *
from functions.PowerCalculation import *
from functions.Resample import *
from functions.StopPointDetection import *
from functions.Elevation import getElevation
#from functions.Preprocessing import * 

In [ ]:
import pymongo

class tag:
    def __init__(self, numPlate):
        self.numPlate = numPlate
    def getLatTag(self):
        return self.numPlate + "_lat"
    def getVelocityTag(self):
        return self.numPlate + "_V"
    def getLngTag(self):
        return self.numPlate + "_lng"
    def getNumPlate(self):
        return self.numPlate
    
car = tag("KF44LNGP")

def latCollection(date, lat, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],
            "latitude": lat[i]
        }
        
        col.append(temp)
    return col

def velCollection(date, velocity, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],           
            "velocity": int(velocity[i])
        }
        
        col.append(temp)
    return col

def lngCollection(date, lng, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],
            "longitude": lng[i]
        }
        
        col.append(temp)
    return col


In [6]:
spark = SparkSession.builder.appName("App").getOrCreate()

In [7]:
data_dir = 'D:\\Telematics\\Car Track\\Data\\Joined'
file = os.path.join(data_dir, os.listdir(data_dir)[0])
df = spark.read.csv(file, inferSchema=True, header=True)
# Test one trip
#df = spark.read.csv(os.path.join(data_dir, os.listdir(data_dir)[0]), inferSchema=True, header=True, sep=";")
df_clean = df.drop("Events", "Vision Events", "Geofences", "Location", "Odometer", "Linear G", "Lateral G", "Road Speed").dropDuplicates().sort("Date").withColumnRenamed("Speed", "velocity")

In [8]:
df_update = df_clean.repartition(3)
df_update.rdd.getNumPartitions()

3

In [12]:
df_update = df_update.withColumn("Partitions", spark_partition_id())
x = df_update.select("velocity","Partitions").distinct()
x.show(n=1000)

+--------+----------+
|velocity|Partitions|
+--------+----------+
|      88|         0|
|      30|         0|
|     141|         0|
|       1|         0|
|      71|         0|
|     119|         0|
|      83|         0|
|     104|         0|
|      54|         0|
|      93|         0|
|     123|         0|
|      56|         0|
|      42|         0|
|      41|         0|
|      64|         0|
|     129|         0|
|      29|         0|
|      95|         0|
|      68|         0|
|       6|         0|
|      59|         0|
|      23|         0|
|     127|         0|
|      61|         0|
|     107|         0|
|       4|         0|
|      26|         0|
|     122|         0|
|     124|         0|
|      16|         0|
|     144|         0|
|     135|         0|
|      39|         0|
|      63|         0|
|     101|         0|
|     118|         0|
|      86|         0|
|     111|         0|
|     108|         0|
|      46|         0|
|     120|         0|
|      84|         0|
|     121|

In [ ]:
group = df_clean.withColumn("Minute", minute(df_clean.Date)).groupBy('Minute', window("Date", "10 seconds")).agg(
    mean("velocity").alias("velocity"),
    mean("Longitude").alias("Longitude"),
    mean("Latitude").alias("Latitude"),
    expr("collect_list(velocity)").alias("velocity list")
)

df_resampled = group.withColumn("Start", group.window.start).withColumn("End", group.window.end).sort("Start")
df_resampled = df_resampled.withColumn("Date",from_unixtime((unix_timestamp(df_resampled.Start) + unix_timestamp(df_resampled.End))/2, "yyyy-MM-dd HH:mm:ss"))
v_clean = df_resampled.select(["Date", "Start", "End", "Velocity", "Latitude", "Longitude"]).withColumn("Date", to_timestamp(df_resampled.Date, "yyyy-MM-dd HH:mm:ss"))



In [ ]:
time, velocity = getTimeVelocity(v_clean)
latitude, longitude = getCoordinates(v_clean)
time_array = getTimestampArr(time)
print(f'Time: {len(time)}  Velocity: {len(velocity)}')
tGaps, indices = timeGaps(time_array, 10)

stime, svelocity, slat, slng = resampleAll(time_array, velocity, latitude, longitude)
#stime, svelocity = resampleTimeVelocity(time_array, velocity)

#print(f'Time: {len(stime)}  Velocity: {len(svelocity)}')
print(f'Time: {len(stime)}  Velocity: {len(svelocity)} Latitude: {len(latitude)} Longitude: {len(longitude)}')



insertLat = latCollection(getDateTime(stime), slat, car.getLatTag())
insertLng = lngCollection(getDateTime(stime), slng, car.getLngTag())



fleet1.insert_many(insertLat)
fleet1.insert_many(insertLng)

In [ ]:
plotTimeSeries(getDateTime(stime), svelocity)

In [ ]:
plotTimeSeries(getDateTime(stime), slat)

In [ ]:
plotTimeSeries(getDateTime(stime), slng)

In [ ]:
insertVel = velCollection(date, velocity, car.getVelocityTag())

In [ ]:
v_clean.show()

In [ ]:
time, velocity = getTimeVelocity(v_clean)
plotTimeSeries(time, velocity)

In [ ]:
np.unique(np.diff(stime))

In [ ]:
plotTimeSeries(getDateTime(stime), svelocity)

In [ ]:

insertLat = latCollection(getDateTime(stime), slat, car.getLatTag())
insertLng = lngCollection(getDateTime(stime), slng, car.getLngTag())
insertVel = velCollection(getDateTime(stime), svelocity, car.getVelocityTag())


fleet1.insert_many(insertLat)
fleet1.insert_many(insertLng)

In [ ]:
client = pymongo.MongoClient()

db = client['fleet']

fleet1 = db['CarTrackProcessed']

car = tag("KF44LNGP")

In [ ]:
fleet1.insert_many(insertVel)

In [ ]:
N = len(stime)
i = 0
SP = []
while i < N:
    Token = 0
    if svelocity[i] < 5:
        start = i
        j = i + 1
        Token = 0
        while j < N:
            if svelocity[j] > 5:
                if stime[j] - stime[i] > 300:
                    end = j - 1
                    t_arrive = stime[start]
                    t_depart = stime[end]
                    SP.append([t_arrive, t_depart, t_depart-t_arrive, [start+1, end-1]])
                    i = j
                    Token = 1
                break
            else:
                j += 1
    if Token != 1:
        i +=1
        

In [ ]:
stopIndex = [x[3] for x in SP]

In [ ]:
def getIndexSlice(StopPoints):
    startStop = []
    for i in range(len(StopPoints)-1):
        if i == 0:
            if StopPoints[i][0] < 6:
                start = StopPoints[i][1]
                end = StopPoints[i+1][0]
                startStop.append((start,end))
            else:
                start = 0
                end = StopPoints[i][0]
                startStop.append((start,end))
                startStop.append((StopPoints[i][1],StopPoints[i+1][0] ))
                
        else:
            startStop.append((StopPoints[i][1],StopPoints[i+1][0] ))
            
    return startStop

for s,f in startStop:
    time_ = time[s:f]
    vel_ = velocity[s:f]
    


In [ ]:
stopIndex

In [ ]:
#plotFigure(df_clean)
plotMap(df_clean)

In [ ]:
days = [x.YMD for x in df_clean.select("Date").withColumn("YMD", date_format(df_clean.Date, "yyyy MM dd")).drop("Date").distinct().sort("YMD").collect()]

In [ ]:
days_array = []
for day in days:    
    temp = df_clean.withColumn("YMD", date_format(df_clean.Date, "yyyy MM dd"))
    temp = temp.filter(temp.YMD == day)
    days_array.append(temp.drop("YMD"))


In [ ]:
test_day = days_array[11]
distThresh = 200/1000
tThresh = 5*60
SP = StayPoint_Detection(test_day,distThresh=distThresh, timeThresh=tThresh)

In [ ]:
stop_points = []
for i,day in enumerate(days_array):
    if i != 10:
        print(f'Day: {i}')
        temp = zeroVelocityPoints(day)
        stop_points.append(temp)

In [ ]:
lat_arr = []
long_arr = []
for day in stop_points:
    for stop in day:
        lat_arr.append(stop[0])
        long_arr.append(stop[1])
    

In [ ]:
elevation = getElevation(lat_arr, long_arr)

In [ ]:
elevation

In [ ]:
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [ ]:
lat, lon = getCoordinates(days_array[15])

In [ ]:
plt.plot(long_arr, lat_arr, marker='.', linewidth=0, color ='#126128')
plt.grid(which='major', color="#cccccc")
plt.title("Geo distribution of GPS tracker")

In [ ]:
len(lat_arr)

In [ ]:
D2 = np.zeros((len(lat_arr), 2), dtype='float32')

In [ ]:
D2[:,0] = np.array(lat_arr)

In [ ]:
D2[:,1] = np.array(long_arr)

D2

In [ ]:
D2

In [ ]:
dbscan_scaler = StandardScaler().fit(D2)
dbscan_data = dbscan_scaler.transform(D2)
dbscan_data

In [ ]:
model = DBSCAN(eps= 0.25, min_samples=12, metric='euclidean').fit(dbscan_data)

In [ ]:
model

In [ ]:
outliers = 

In [ ]:
group = df_clean.withColumn("Minute", minute(df_clean.Date)).groupBy('Minute', window("Date", "10 seconds")).agg(
    mean("velocity").alias("velocity"), 
    expr("collect_list(velocity)").alias("velocity list")
)

df_resampled = group.withColumn("Start", group.window.start).withColumn("End", group.window.end).sort("Start")
df_resampled = df_resampled.withColumn("Date",from_unixtime((unix_timestamp(df_resampled.Start) + unix_timestamp(df_resampled.End))/2, "yyyy-MM-dd HH:mm:ss"))

In [ ]:
v_clean = df_resampled.select(["Date", "Start", "End", "velocity"]).withColumn("Date", to_timestamp(df_resampled.Date, "yyyy-MM-dd HH:mm:ss"))

In [ ]:
time, velocity = getTimeVelocity(v_clean)
time_array = getTimestampArr(time)

tGaps, indices = timeGaps(time_array, 10)

In [ ]:
getTimeVelocity(v_clean)

In [ ]:
time, velocity = getTimeVelocity(v_clean)
time_array = getTimestampArr(time)

tGaps, indices = timeGaps(time_array, 10)

for tg in tGaps:
    i = getNextTimeGap(time_array, 10)
    start = time_array[i]
    end = time_array[i+1]
    temp = start
    row = []
    for j in range(int(tg)-1):
        temp += 10
        row.append(temp)
        
    insertList(time_array, i+1, row)
    insertVelocity(velocity, i, tg)
    

In [ ]:
len(velocity)

In [ ]:
len(time_array)

In [ ]:
_, acc = central_difference(time_array, velocity)

In [ ]:
fig = go.Figure([go.Scatter(x=time, y=velocity)])
fig.show()


fig = go.Figure([go.Scatter(x=time_array, y=acc)])
fig.show()

In [ ]:
_, power = getPower(1300, acc, velocity, 0.02, 0.33, 1.75*1.48, theta=np.radians(5))

In [ ]:
fig = go.Figure([go.Scatter(x=[datetime.fromtimestamp(a) for a in time_array], y=power)])
fig.show()

In [ ]:
energy = getTotalEnergy(time_array, power)

In [ ]:
lat, long = getCoordinates(df_clean)
distance = getTotalDistance(lat, long)

In [ ]:
EPK(energy, distance)

In [ ]:
getDistance(lat[0], long[0], lat[1], long[1])

In [ ]:
distance

In [ ]:
df_clean.show()

In [ ]:
df_clean.describe().show()

In [ ]:
import requests

In [ ]:
r = requests.get('https://api.open-elevation.com/api/v1/lookup?locations=10,10|20,20|41.161758,-8.583933')
#payload = {"locations":["26.13467 ,  28.229599]}

print(r.status_code)

In [ ]:
payload = location
base_url = 'https://api.open-elevation.com/api/v1/lookup'
r = requests.get(base_url, params=payload)
r.text

In [ ]:
r.text

In [ ]:
from urllib import request, error, parse
import json
i
d_ar = [{}]* np.shape(D2)[0]

In [ ]:
for i,x in enumerate(D2):
    d_ar[i] = {"latitude":float(x[0]),"longitude":float(x[1])}
d_ar

In [ ]:
location={"locations":d_ar}

In [ ]:
json_data = json.dumps(location, skipkeys=True).encode('utf8')
response = request.Request(base_url, data=json_data, headers={'Content-Type': 'application/json'})

In [ ]:
response

In [ ]:
fp=urllib.request.urlopen(response)

#RESPONSE PROCESSING
res_byte=fp.read()
res_str=res_byte.decode("utf8")
js_str=json.loads(res_str)
fp.close()

In [ ]:
elevation = [x['elevation'] for x in js_str['results']]

In [ ]:
fp=urllib.request.urlopen(response)
res = json.load(fp)
print(res)
fp.close()

In [ ]:
fp